In [0]:
import numpy as np
import pickle, keras
import pandas as pd
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense,Embedding, Dropout, GRU, BatchNormalization, Conv2D, MaxPooling2D, Activation
import matplotlib.pyplot as plt
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# lets load data from pickle file we previously stored
data_file = '/content/drive/My Drive/pickles/train_berkeley_skeleton.pickle' # redefined varaible in case you have completed above steps previously.
print('Trying to load pickle from %s' % data_file)
with open(data_file, 'rb') as file:
    svhn_datasets = pickle.load(file)
    train_dataset = svhn_datasets['train_dataset']
    
print('Pickle Loaded Successfully!')
del svhn_datasets
x_train = train_dataset['X_train']
y_train = train_dataset['Y_train']

In [0]:
# num_img = []
# for i in range(len(x_train)):
#   [m,n] = x_train[i].shape
#   num_img.append(m)
#   print(m)
x_all = pd.concat(x_train).values
Y = pd.concat(y_train).values
print(x_all.shape)
print(Y.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
x_all = scaler.fit_transform(x_all)
X=x_all.reshape(x_all.shape[0], 1, x_all.shape[1])

# Y = np.array(list(map(one_hot_vec, Y)))
Y = keras.utils.to_categorical(Y, num_classes=11)
print(X.shape)
print(Y.shape)

In [0]:
def make_model(lr):
        
    model = Sequential()
    dropout = 0.5
    activation = 'elu'
    model.add(LSTM(units= 64,return_sequences= True, activation=activation, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(dropout))
    model.add(LSTM(units= 64, activation='relu'))
    model.add(Dropout(dropout))
 #   model.add(LSTM(16, activation= 'relu'))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(32, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(11, activation='softmax'))
    opt = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
model = make_model(0.002)

In [0]:
history = model.fit(X, Y,epochs = 10 , batch_size=64)
plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [0]:
data_file = '/content/drive/My Drive/pickles/test_berkeley_skeleton.pickle' # redefined varaible in case you have completed above steps previously.
print('Trying to load pickle from %s' % data_file)
with open(data_file, 'rb') as file:
    svhn_datasets = pickle.load(file)
    test_dataset = svhn_datasets['test_dataset']
    
print('Pickle Loaded Successfully!')
del svhn_datasets
x_test = test_dataset['X_test']
y_test = test_dataset['Y_test']

x_all = pd.concat(x_test).values
Y = pd.concat(y_test).values
print(x_all.shape)
print(Y.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
x_all = scaler.fit_transform(x_all)
X=x_all.reshape(x_all.shape[0], 1, x_all.shape[1])

# Y = np.array(list(map(one_hot_vec, Y)))
Y = keras.utils.to_categorical(Y, num_classes=11)
print(X.shape)
print(Y.shape)

In [0]:
scores = model.evaluate(X, Y, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

In [0]:
model_json = model.to_json()
with open('drive/My Drive/models/rnn_6.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('drive/My Drive/models/rnn_6.h5')